In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS
from statsmodels.tools import add_constant

In [2]:
def regressHart(y, X, Z, **kwargs):
    n = len(y)
    if n != X.shape[0]:
        raise ValueError('Size mismatch')
    maxiterdef = 10000
    toldef = 1e-08
    test=0
    options = dict(
        intercept=True,
        maxiter=maxiterdef,
        type='art',
        initialbeta=None, # shall be array of shape (n_features, ) for single target regression
        initialgamma=None,
        tol=toldef,
        nocheck=False,
        msgiter=0,
        test=test
    )
    test = options['test']
    reg = LinearRegression(fit_intercept=options['intercept'])
    # number of predictors
    if options['intercept']:
        p = X.shape[1] + 1
    else:
        p = X.shape[1]
    reg_results0 = reg.fit(X, y)
    b0 = np.c_[reg_results0.intercept_, reg_results0.coef_] # b0 is array of shape (n_features, ) for single target regression
    r = y - reg_results0.predict(X)
    oldbeta = b0
    sigma2 = r.T @ r / (n)
    # loglikelihood of residual
    logL_R = n * (1 + np.log(sigma2))
    #  Initialization of gamma
    #  Z = n-by-r matrix which contains the explanatory variables for
    #  heteroskedasticity
    Z_ = Z.copy
    # if options['intercept']:
    #     Z_ = add_constant(Z_)
    response_gamma0 = n * np.square(r) / np.sum(np.square(r)) - 1
    reg_gamma = reg.fit(Z, response_gamma0)
    gamma0 = np.c_[reg_gamma.intercept_, reg_gamma.coef_]
    # gamma
    oldgamma = gamma0
    # tolerance
    tol = options['tol']
    cont = 1
    iter = 0
    maxiter = options['maxiter']
    delt = 1
    th = 8
    dold = np.c_[oldbeta, oldgamma]
    print(f'{dold}')
    while (cont == 1) & (iter < maxiter):
        iter = iter + 1
        if options['intercept']:
            Zoldgamma = (np.c_[np.repeat(1, n), Z] @ oldgamma.T).flatten()
        Zoldgamma[Zoldgamma < -th] = -th
        Zoldgamma[Zoldgamma > th] = th
        expZgamma = np.exp(Zoldgamma)
        omegahat = 1 + expZgamma
        sqrtweight = np.sqrt(omegahat)
        reg_new = reg.fit(X, y, sample_weight=omegahat)
        y_new = reg_new.predict(X)
        newbeta = np.c_[reg_new.intercept_, reg_new.coef_]
        newres2 = np.square(sqrtweight * (y - y_new))
        newsigma2 = np.sum(newres2) / (n)
        Qweights = expZgamma / (1 + expZgamma)
        Zq = Z * Qweights.reshape(-1, 1)
        newres2ori = y - y_new
        yq = newres2ori / (newsigma2 * (1 + expZgamma))
        reg_q = reg.fit(Zq, yq)
        gamma_q = np.c_[reg_q.intercept_, reg_q.coef_]
        newgamma = oldgamma + gamma_q * delt
        dnew = np.c_[newbeta, newgamma]
        print(f'{dnew}')
        if (np.sum(np.square(dnew - dold)) / np.sum(np.square(dold))) > tol:
            cont = 1
            oldgamma = newgamma
            dold=dnew
        else:
            cont = 0
    return y_new

In [3]:
def regressHart(y, X, Z, **kwargs):
    n = len(y)
    if n != X.shape[0]:
        raise ValueError('Size mismatch')
    maxiterdef = 10000
    toldef = 1e-12
    test=0
    options = dict(
        intercept=True,
        maxiter=maxiterdef,
        type='art',
        initialbeta=None, # shall be array of shape (n_features, ) for single target regression
        initialgamma=None,
        tol=toldef,
        nocheck=False,
        msgiter=0,
        test=test
    )
    test = options['test']
    reg = LinearRegression(fit_intercept=options['intercept'])
    # number of predictors
    if options['intercept']:
        p = X.shape[1] + 1
    else:
        p = X.shape[1]
    reg_results0 = reg.fit(X, y)
    b0 = np.c_[reg_results0.intercept_, reg_results0.coef_] # b0 is array of shape (n_features, ) for single target regression
    r = y - reg_results0.predict(X)
    oldbeta = b0
    sigma2 = r.T @ r / (n)
    # loglikelihood of residual
    logL_R = n * (1 + np.log(sigma2))
    #  Initialization of gamma
    #  Z = n-by-r matrix which contains the explanatory variables for
    #  heteroskedasticity
    Z_ = Z.copy
    # if options['intercept']:
    #     Z_ = add_constant(Z_)
    response_gamma0 = n * np.square(r) / np.sum(np.square(r)) - 1
    reg_gamma = reg.fit(Z, response_gamma0)
    gamma0 = np.c_[reg_gamma.intercept_, reg_gamma.coef_]
    # gamma
    oldgamma = gamma0
    # tolerance
    tol = options['tol']
    cont = 1
    iter = 0
    maxiter = options['maxiter']
    delt = 0.5
    th = 8
    dold = np.c_[oldbeta, oldgamma]
    print(f'{dold}')
    while (cont == 1) & (iter < maxiter):
        iter = iter + 1
        if options['intercept']:
            Zoldgamma = (np.c_[np.repeat(1, n), Z] @ oldgamma.T).flatten()
        # Zoldgamma[Zoldgamma < -th] = -th
        # Zoldgamma[Zoldgamma > th] = th
        expZgamma = np.exp(Zoldgamma)
        omegahat = 1 + expZgamma
        weight = np.power(omegahat, -1)
        sqrtweight = np.sqrt(weight)
        reg_new = reg.fit(X, y, sample_weight=weight)
        y_new = reg_new.predict(X)
        newbeta = np.c_[reg_new.intercept_, reg_new.coef_]
        newres2 = np.square(sqrtweight * (y - y_new))
        newsigma2 = np.sum(newres2) / (n)
        Qweights = expZgamma / (1 + expZgamma)
        Zq = Z * Qweights.reshape(-1, 1)
        newres2ori = y - y_new
        yq = newres2ori / (newsigma2 * (1 + expZgamma))
        reg_q = reg.fit(Zq, yq)
        gamma_q = np.c_[reg_q.intercept_, reg_q.coef_]
        newgamma = oldgamma + gamma_q * delt
        dnew = np.c_[newbeta, newgamma]
        print(f'{dnew}')
        if (np.sum(np.square(dnew - dold)) / np.sum(np.square(dold))) > tol:
            cont = 1
            oldgamma = newgamma
            dold=dnew
        else:
            cont = 0
    return y_new

In [4]:
regressHart(y, X, X)

NameError: name 'y' is not defined

In [ ]:
np.c_[np.repeat(1, 1100), X] @ np.array([1, 2])

In [ ]:
trade_df = pd.read_csv('TradeH.csv', sep='\t', header=None, names=['quantity', 'value'])

X = trade_df['quantity'].values.reshape(-1, 1)
y = trade_df['value'].values

In [ ]:
type(regressHart(y, X, X))

In [ ]:
regressHart(y, X, X)[np.argsort(X.flatten)]

In [ ]:
a = regressHart(y, X, X)
a

In [ ]:
X.flatten()

In [ ]:
import matplotlib.pyplot as plt

reg = LinearRegression()
reg_result = reg.fit(X, y)


fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(np.sort(X.flatten()), regressHart(y, X, X)[np.argsort(X.flatten())], color='red')

In [ ]:
import matplotlib.pyplot as plt

reg = LinearRegression()
reg_result = reg.fit(X, y)


fig, ax = plt.subplots(figsize=(12, 12))

ax.scatter(x=X, y=y)
ax.plot(np.sort(X.flatten()), regressHart(y, X, X)[np.argsort(X.flatten())], color='red')
ax.plot(np.sort(X.flatten()), reg_result.predict(X)[np.argsort(X.flatten())], color='black')

In [ ]:
X_df = pd.read_csv('X_train.csv')
y_df = pd.read_csv('y_train.csv')


X = X_df['building_size_m2'].values.reshape(-1, 1)
y = y_df['price_in_rp'].values

In [ ]:
weight

In [ ]:
import matplotlib.pyplot as plt

X_df = pd.read_csv('X_train.csv')
y_df = pd.read_csv('y_train.csv')


X = X_df['building_size_m2'].values.reshape(-1, 1)
y = y_df['price_in_rp'].values

reg = LinearRegression()
reg_result = reg.fit(X, y)


fig, ax = plt.subplots(figsize=(12, 12))
weighted_y = weight * 2.5e11
ax.scatter(x=X, y=y)
ax.plot(np.sort(X.flatten()), regressHart(y, X, X)[np.argsort(X.flatten())], color='red')
ax.plot(np.sort(X.flatten()), reg_result.predict(X)[np.argsort(X.flatten())], color='black')
ax.plot(np.sort(X.flatten()), weighted_y[np.argsort(X.flatten())], color='yellow')

In [ ]:
min(y)

In [ ]:
def regressHart(y, X, Z, **kwargs):
    n = len(y)
    if n != X.shape[0]:
        raise ValueError('Size mismatch')
    maxiterdef = 10000
    toldef = 1e-12
    test=0
    options = dict(
        intercept=True,
        maxiter=maxiterdef,
        type='art',
        initialbeta=None, # shall be array of shape (n_features, ) for single target regression
        initialgamma=None,
        tol=toldef,
        nocheck=False,
        msgiter=0,
        test=test
    )
    test = options['test']
    reg = LinearRegression(fit_intercept=options['intercept'])
    # number of predictors
    if options['intercept']:
        p = X.shape[1] + 1
    else:
        p = X.shape[1]
    reg_results0 = reg.fit(X, y)
    b0 = np.c_[reg_results0.intercept_, reg_results0.coef_] # b0 is array of shape (n_features, ) for single target regression
    r = y - reg_results0.predict(X)
    oldbeta = b0
    sigma2 = r.T @ r / (n)
    # loglikelihood of residual
    logL_R = n * (1 + np.log(sigma2))
    #  Initialization of gamma
    #  Z = n-by-r matrix which contains the explanatory variables for
    #  heteroskedasticity
    Z_ = Z.copy
    # if options['intercept']:
    #     Z_ = add_constant(Z_)
    response_gamma0 = n * np.square(r) / np.sum(np.square(r)) - 1
    reg_gamma = reg.fit(Z, response_gamma0)
    gamma0 = np.c_[reg_gamma.intercept_, reg_gamma.coef_]
    # gamma
    oldgamma = gamma0
    # tolerance
    tol = options['tol']
    cont = 1
    iter = 0
    maxiter = options['maxiter']
    delt = 0.5
    th = 8
    dold = np.c_[oldbeta, oldgamma]
    print(f'{dold}')
    while (cont == 1) & (iter < maxiter):
        iter = iter + 1
        if options['intercept']:
            Zoldgamma = (np.c_[np.repeat(1, n), Z] @ oldgamma.T).flatten()
        # Zoldgamma[Zoldgamma < -th] = -th
        # Zoldgamma[Zoldgamma > th] = th
        expZgamma = np.exp(Zoldgamma)
        omegahat = 1 + expZgamma
        weight = np.power(omegahat, -1)
        sqrtweight = np.sqrt(weight)
        reg_new = reg.fit(X, y, sample_weight=weight)
        y_new = reg_new.predict(X)
        newbeta = np.c_[reg_new.intercept_, reg_new.coef_]
        newres2 = np.square(sqrtweight * (y - y_new))
        newsigma2 = np.sum(newres2) / (n)
        Qweights = expZgamma / (1 + expZgamma)
        Zq = Z * Qweights.reshape(-1, 1)
        newres2ori = y - y_new
        yq = newres2ori / (newsigma2 * (1 + expZgamma))
        reg_q = reg.fit(Zq, yq)
        gamma_q = np.c_[reg_q.intercept_, reg_q.coef_]
        newgamma = oldgamma + gamma_q * delt
        dnew = np.c_[newbeta, newgamma]
        print(f'{dnew}')
        if (np.sum(np.square(dnew - dold)) / np.sum(np.square(dold))) > tol:
            cont = 1
            oldgamma = newgamma
            dold=dnew
        else:
            cont = 0
    return weight

In [ ]:
import matplotlib.pyplot as plt

reg = LinearRegression()
reg_result = reg.fit(X, y)


fig, ax = plt.subplots(figsize=(12, 6))
weight = regressHart(y, X, X)
ax.plot(np.sort(X.flatten()), regressHart(y, X, X)[np.argsort(X.flatten())], color='red')

In [ ]:
def regressHart(y, X, Z, **kwargs):
    n = len(y)
    if n != X.shape[0]:
        raise ValueError('Size mismatch')
    maxiterdef = 10000
    toldef = 1e-12
    test=0
    options = dict(
        intercept=True,
        maxiter=maxiterdef,
        type='art',
        initialbeta=None, # shall be array of shape (n_features, ) for single target regression
        initialgamma=None,
        tol=toldef,
        nocheck=False,
        msgiter=0,
        test=test
    )
    test = options['test']
    reg = LinearRegression(fit_intercept=options['intercept'])
    # number of predictors
    if options['intercept']:
        p = X.shape[1] + 1
    else:
        p = X.shape[1]
    reg_results0 = reg.fit(X, y)
    b0 = np.c_[reg_results0.intercept_, reg_results0.coef_] # b0 is array of shape (n_features, ) for single target regression
    r = y - reg_results0.predict(X)
    oldbeta = b0
    sigma2 = r.T @ r / (n)
    # loglikelihood of residual
    logL_R = n * (1 + np.log(sigma2))
    #  Initialization of gamma
    #  Z = n-by-r matrix which contains the explanatory variables for
    #  heteroskedasticity
    Z_ = Z.copy
    # if options['intercept']:
    #     Z_ = add_constant(Z_)
    response_gamma0 = n * np.square(r) / np.sum(np.square(r)) - 1
    reg_gamma = reg.fit(Z, response_gamma0)
    gamma0 = np.c_[reg_gamma.intercept_, reg_gamma.coef_]
    # gamma
    oldgamma = gamma0
    # tolerance
    tol = options['tol']
    cont = 1
    iter = 0
    maxiter = options['maxiter']
    delt = 0.5
    th = 8
    dold = np.c_[oldbeta, oldgamma]
    print(f'{dold}')
    while (cont == 1) & (iter < maxiter):
        iter = iter + 1
        if options['intercept']:
            Zoldgamma = (np.c_[np.repeat(1, n), Z] @ oldgamma.T).flatten()
        # Zoldgamma[Zoldgamma < -th] = -th
        # Zoldgamma[Zoldgamma > th] = th
        expZgamma = np.exp(Zoldgamma)
        omegahat = 1 + expZgamma
        weight = np.power(omegahat, -1)
        sqrtweight = np.sqrt(weight)
        reg_new = reg.fit(X, y, sample_weight=weight)
        y_new = reg_new.predict(X)
        newbeta = np.c_[reg_new.intercept_, reg_new.coef_]
        newres2 = np.square(sqrtweight * (y - y_new))
        newsigma2 = np.sum(newres2) / (n)
        Qweights = expZgamma / (1 + expZgamma)
        Zq = Z * Qweights.reshape(-1, 1)
        newres2ori = y - y_new
        yq = newres2ori / (newsigma2 * (1 + expZgamma))
        reg_q = reg.fit(Zq, yq)
        gamma_q = np.c_[reg_q.intercept_, reg_q.coef_]
        newgamma = oldgamma + gamma_q * delt
        dnew = np.c_[newbeta, newgamma]
        print(f'{dnew}')
        if (np.sum(np.square(dnew - dold)) / np.sum(np.square(dold))) > tol:
            cont = 1
            oldgamma = newgamma
            dold=dnew
        else:
            cont = 0
    return reg_new.predict(np.array([500]).reshape(-1, 1))

In [ ]:
y

In [ ]:
regressHart(y, X, X)

In [6]:
int(True)

1

In [7]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_is_fitted

class RegressHART(BaseEstimator, RegressorMixin):
    def __init__(self, fit_intercept=True, max_iter=1000, 
            initialbeta=None, initialgamma=None, tol=1e-8):
        self.fit_intercept = fit_intercept
        self.initialbeta = initialbeta
        self.initialgamma = initialgamma
        self.max_iter = max_iter
        self.tol = tol

    def _estimate_gamma(self, X, y, Z):
        """ Estimate gamma for the parametric weight"""
        reg = LinearRegression(fit_intercept=self.fit_intercept)
        reg_result0 = reg.fit(X, y)
        if self.fit_intercept:
            self._n_params = reg_result0.n_features_in_ + 1
        else:
            self._n_params = reg_result0.n_features_in_
        # since ._validate_data() has been performed by sklearn estimator,
        # length for response (n_samples, ) has been checked to match with
        # regressor (n_samples, n_features), so:
        self._n_samples = len(y)
        residual = y - reg_result0.predict(X)
        sigma2 = \
            residual.T @ residual / (self._n_samples - self._n_params) # estimate of variance
        # initial estimate of gamma, using Z and 
        #
        response_gamma0 = \
            self._n_samples * np.square(residual) / np.sum(np.square(residual)) - 1
        # Shape of Z (n_samples, n_heteroscedastic_features)
        # will also be verified by sklearn estimator to match with 
        # response y (n_samples, )
        reg_gamma = reg.fit(Z, response_gamma0)
        if self.fit_intercept:
            return reg_gamma.intercept_, reg_gamma.coef_
        return reg_gamma.coef_

    def _weighted_regressor(self, X, y, Z, gamma):
        if self.fit_intercept:
            zgamma = (np.c_[np.repeat(1, self._n_samples), Z] @ gamma.T).flatten()
        else:
            zgamma = (Z @ gamma.T).flatten()
        expzgamma = np.exp(zgamma)
        weight = np.power((1 + expzgamma), -1)
        reg = self._reg.fit(X, y, sample_weight=weight)
        return reg
    
    def _scoring(self, X, y, Z, fitted_regressor, gamma):
        if self.fit_intercept:
            zgamma = (np.c_[np.repeat(1, self._n_samples), Z] @ gamma.T).flatten()
        else:
            zgamma = (Z @ gamma.T).flatten()
        y_predict = fitted_regressor.predict(X)
        expzgamma = np.exp(zgamma)
        weight = np.power((1 + expzgamma), -1)
        sqrtweight = np.sqrt(weight)
        weighted_res2 = np.square(sqrtweight * (y - y_predict))
        sigma2 = np.sum(weighted_res2) / (self._n_samples - self._n_params)
        q = expzgamma / (1 + expzgamma)
        zq = Z * q.reshape(-1, 1)
        res2 = np.square(y - y_predict)
        yq = res2 / (sigma2 * (1 + expzgamma))
        reg_q = self._reg.fit(zq, yq)
        gamma_q = np.c_[reg_q.intercept_, reg_q.coef_]
        return gamma_q

    def fit(self, X, y, Z):
        # since we wrap sklearn function, ._validate_data() will be
        # performed under sklearn estimator
        self._n_params = None
        self._n_samples = None
        self._reg = LinearRegression(fit_intercept=self.fit_intercept)
        if self.initial_gamma:
            gamma = self.initial_gamma
        else:
            if self.fit_intercept:
                gamma_intercept, gamma_coef = self._estimate_gamma(X, y, Z)
                gamma = np.c_[gamma_intercept, gamma_coef]
            else:
                gamma = self._estimate_gamma(X, y, Z)
        # initiate iter k + 1 until tolerance satisfied
        # or max_iter excedeed
        gamma_arr = gamma
        reg_result_arr = np.array([self._weighted_regressor(X, y, Z, gamma)])
        if self.fit_intercept:
            reg_params_arr = \
                np.c_[np.array(reg_result_arr[0].intercept_), reg_result_arr[0].coef_]
        else:
            reg_params_arr = reg_result_arr[0].coef_
        d_arr = ...
        iter = 1
        delt = 0.9
        for i in range(self.max_iter):
            gamma_q = self._scoring(X, y, Z, reg_result_arr[i], gamma_arr[i])
            gamma_arr = np.r_[gamma_arr, (gamma_arr[i] + delt * gamma_q)]\
                .reshape(i + 2, gamma_arr[i].shape[0])
            reg_result_arr = \
                np.r_[reg_result_arr, self._weighted_regressor(X, y, Z, gamma_arr[i + 1])]\
                    .reshape(-1, 1)
            reg_params_arr = \
                np.r_[
                    reg_params_arr,
                    np.c_[np.array(reg_result_arr[i + 1].intercept_), 
                        reg_result_arr[i + 1].coef_]
                ].reshape(i + 2, reg_params_arr[0].shape[1])
            d_new = ...
            d_arr = np.r_[d_arr, d_new]
            tol = np.sum(np.square(d_arr[i + 1] - d_arr[i])) / np.sum(np.square(d_arr[i]))
            if tol < self.tol:
                break
        # Final converged parameter
        gamma = gamma_arr[-1]
        if self.fit_intercept:
            zgamma = (np.c_[np.repeat(1, self._n_samples), Z] @ gamma.T).flatten()
        else:
            zgamma = (Z @ gamma.T).flatten()   
        expzgamma = np.exp(zgamma)
        weight = np.power((1 + expzgamma), -1)    
        self._reg_result = reg_result_arr[-1]
        self.coef_ = self.reg_result.coef_
        self.intercept_ = self.reg_result.intercept_
        self.n_features_in_ = self.reg_result.n_features_in
        self.feature_names_in_ = self.reg_result.feature_names_in_
        self.gamma_ = gamma_arr
        self.weight_ = weight
        self.n_iter_ = iter
        return self      

    def predict(self, X):
        check_is_fitted(self)
        return self._reg_result.predict(X)




        while (iter < self.max_iter) & (tol > self.tol):
            # calculate weight at k = 1 (inital iteration)
            # k = 1 regression weight is based on provided or estimated inital gamma
            # also store iterated gamma and weight
            expzgamma = np.exp(zgamma)
            weight = np.power((1 + expzgamma), -1)
            # initial regression for k = 1 using estimated weight
            # we also store iterated coefficient
            self._reg_result = self._reg.fit(X, y, sample_weight=weight)
            if self.fit_intercept:
                reg_params = np.c_[np.array(self._reg_result.intercept_), self._reg_result.coef_]
            else:
                reg_params = self._reg_result.coef_
            if iter == 1:
                self.iterated_params_ = reg_params
            else:
                self.iterated_params_ = np.r_[self.iterated_params_, reg_params]
            # initiate scoring for gamma
            gamma_q = self._scoring(X, y, Z, self._reg_result, zgamma)
            delta = 0.8
            gamma_new = gamma +  gamma_q * delta
            # properties of k + 1
            zgamma_new = (Z @ gamma_new.T).flatten()
            expzgamma_new = np.exp(zgamma_new)
            weight = np.power((1 + expzgamma_new), -1)
            reg_params_new = 
            # iteration tolerance
            d = np.c_[reg_params, gamma]
            d_new = 
            tol = np.sum(np.square(d_new - d)) / np.sum(np.square(d))
            # if tol is not satisfied, loop will be performed,
            # gamma will be replaced by scoring algorithm result
            # so that first line of the loop will consider new gamma
            # instead of estimated gamma
            if tol > self.tol:
                gamma = gamma_new
                zgamma = (Z @ gamma.T).flatten()
                self.iterated_gamma = np.r_[self.iterated_gamma_, gamma_new]
                iter += 1
        self.coef_ = reg_result.coef_
        self.intercept_ = reg_result.intercept_
        self.n_features_in_ = reg_result.n_features_in
        self.feature_names_in_ = reg_result.feature_names_in_
        self.gamma_ = gamma
        self.weight_ = weight
        self.n_iter_ = iter
        return self
    


        
        
    

SyntaxError: invalid syntax (187259090.py, line 98)

In [21]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c = np.array([7, 8, 9])

d = np.append(a, b).reshape(2, 3)
e = np.append(d, c).reshape(3, 3)

a.shape[0]




3

In [9]:
a = np.array([1, 2])

np.r_[a, 1]

array([1, 2, 1])

In [ ]:
def regressHart(y, X, Z, **kwargs):
    n = len(y)
    if n != X.shape[0]:
        raise ValueError('Size mismatch')
    maxiterdef = 10000
    toldef = 1e-12
    test=0
    options = dict(
        intercept=True,
        maxiter=maxiterdef,
        type='art',
        initialbeta=None, # shall be array of shape (n_features, ) for single target regression
        initialgamma=None,
        tol=toldef,
        nocheck=False,
        msgiter=0,
        test=test
    )
    test = options['test']
    reg = LinearRegression(fit_intercept=options['intercept'])
    # number of predictors
    if options['intercept']:
        p = X.shape[1] + 1
    else:
        p = X.shape[1]
    reg_results0 = reg.fit(X, y)
    b0 = np.c_[reg_results0.intercept_, reg_results0.coef_] # b0 is array of shape (n_features, ) for single target regression
    r = y - reg_results0.predict(X)
    oldbeta = b0
    sigma2 = r.T @ r / (n)
    # loglikelihood of residual
    logL_R = n * (1 + np.log(sigma2))
    #  Initialization of gamma
    #  Z = n-by-r matrix which contains the explanatory variables for
    #  heteroskedasticity
    Z_ = Z.copy
    # if options['intercept']:
    #     Z_ = add_constant(Z_)
    response_gamma0 = n * np.square(r) / np.sum(np.square(r)) - 1
    reg_gamma = reg.fit(Z, response_gamma0)
    gamma0 = np.c_[reg_gamma.intercept_, reg_gamma.coef_]
    # gamma
    oldgamma = gamma0
    # tolerance
    tol = options['tol']
    cont = 1
    iter = 0
    maxiter = options['maxiter']
    delt = 0.5
    th = 8
    dold = np.c_[oldbeta, oldgamma]
    print(f'{dold}')
    while (cont == 1) & (iter < maxiter):
        iter = iter + 1
        if options['intercept']:
            Zoldgamma = (np.c_[np.repeat(1, n), Z] @ oldgamma.T).flatten()
        # Zoldgamma[Zoldgamma < -th] = -th
        # Zoldgamma[Zoldgamma > th] = th
        expZgamma = np.exp(Zoldgamma)
        omegahat = 1 + expZgamma
        weight = np.power(omegahat, -1)
        sqrtweight = np.sqrt(weight)
        reg_new = reg.fit(X, y, sample_weight=weight)
        y_new = reg_new.predict(X)
        newbeta = np.c_[reg_new.intercept_, reg_new.coef_]
        newres2 = np.square(sqrtweight * (y - y_new))
        newsigma2 = np.sum(newres2) / (n)
        Qweights = expZgamma / (1 + expZgamma)
        Zq = Z * Qweights.reshape(-1, 1)
        newres2ori = y - y_new
        yq = newres2ori / (newsigma2 * (1 + expZgamma))
        reg_q = reg.fit(Zq, yq)
        gamma_q = np.c_[reg_q.intercept_, reg_q.coef_]
        newgamma = oldgamma + gamma_q * delt
        dnew = np.c_[newbeta, newgamma]
        print(f'{dnew}')
        if (np.sum(np.square(dnew - dold)) / np.sum(np.square(dold))) > tol:
            cont = 1
            oldgamma = newgamma
            dold=dnew
        else:
            cont = 0
    return y_new

In [ ]:
reg = LinearRegression().fit(X, y).predict(X)

reg.coef_ 